In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import dask
from datetime import timedelta
import glob
%matplotlib inline

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap

import matplotlib as mpl
mpl.use('agg')

# import personal modules
import custom_cmaps as ccmap
from plotter import draw_basemap
import mclimate_funcs as mclim_func
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [4]:
%%time
## load all lead times
mon = 11
day = 17

## load mclimate data
mclimate = mclim_func.load_mclimate(mon, day)

step = mclimate.step.values
forecast_lst = []
for i, st in enumerate(step):
    fname = path_to_data + 'downloads/GFS_025d/GFS_IVT_2023111700_F{0}.nc'.format(st)

    ## load the forecast    
    forecast = xr.open_dataset(fname)
    forecast = forecast.rename({'lon_0': 'lon', 'lat_0': 'lat'}) # need to rename this to match GEFSv12 Reforecast
    forecast = forecast.assign_coords({"lon": (((forecast.lon + 180) % 360) - 180)}) # Convert DataArray longitude coordinates from 0-359 to -180-179
    forecast = forecast.sel(lon=slice(-179.5, -110.))
    forecast = forecast.drop_vars(["uIVT", "vIVT"])
    forecast_lst.append(forecast)

forecast = xr.concat(forecast_lst, pd.Index(step, name="step"))

## load the data into memory
mclimate = mclimate.load()
forecast = forecast.load()

## compare the mclimate to the reforecast
ds = mclim_func.compare_mclimate_to_forecast(forecast, mclimate)
ds = ds.assign_attrs(init_time='2023111700')

## create .txt file with max value within Southeast AK
ext_SEAK = [-141., -130., 54.5, 60.] # extent of SEAK 
tmp = ds.sel(lat=slice(ext_SEAK[2], ext_SEAK[3]), lon=slice(ext_SEAK[0], ext_SEAK[1]))
maxivt = tmp.ivt_mclimate.max(dim=['lat', 'lon'])
maxivt

CPU times: user 463 ms, sys: 247 ms, total: 710 ms
Wall time: 2.23 s


<xarray.DataArray 'ivt_mclimate' (step: 40)>
array([0.96, 0.98, 0.98, 0.98, 0.97, 0.96, 0.94, 0.9 , 0.75, 0.75,  nan,
        nan, 0.75, 0.96, 0.99, 0.99, 0.99, 0.99, 0.99, 0.96, 0.93, 0.92,
       0.75, 0.75, 0.75, 0.94, 0.94, 0.75, 0.75, 0.75, 0.75, 0.9 , 0.92,
       0.94, 0.96, 0.96, 0.98, 0.98, 0.98, 0.97])
Coordinates:
  * step     (step) int64 6 12 18 24 30 36 42 48 ... 204 210 216 222 228 234 240

In [5]:
## create list of valid dates
ts = pd.to_datetime(ds.init_time, format="%Y%m%d%H")
init_time = ts.strftime('Initialized: %HZ %d %b %Y')
col2 = []
col3 = []
step_lst = ds.step.values
for i, step in enumerate(step_lst):
    ts_valid = ts + timedelta(hours=int(step))
    col2.append(ts_valid.strftime('%a %d'))
    col3.append(ts_valid.strftime('%HZ'))


## create dataframe
d = {'F': ds.step.values, 
     'Date': col2,
     'Hour': col3,
     'IVT': maxivt.values}

df = pd.DataFrame(d)

def make_clickable(init_date, step, mclimate_val):
    fname = 'images/mclimate/ivt_mclimate_{0}_F{1}.png'.format(init_date, step)
    string_arg = "image.src='{0}'".format(fname)
    return '<a href="#image" onclick="{0}">{1}</a>'.format(string_arg, mclimate_val)
    # return '<a href="#image" onclick="showImage("images/mclimate/ivt_mclimate_{0}_F{1}.png");">{2}</a>'.format(init_date, step, mclimate_val)

df['IVT'] = df.apply(lambda x: make_clickable(ds.init_time, x['F'], x['IVT']), axis=1)
df

,F,Date,Hour,IVT
0,6,Fri 17,06Z,"<a href=""#image"" onclick=""image.src='images/mc..."
1,12,Fri 17,12Z,"<a href=""#image"" onclick=""image.src='images/mc..."
2,18,Fri 17,18Z,"<a href=""#image"" onclick=""image.src='images/mc..."
3,24,Sat 18,00Z,"<a href=""#image"" onclick=""image.src='images/mc..."
4,30,Sat 18,06Z,"<a href=""#image"" onclick=""image.src='images/mc..."
5,36,Sat 18,12Z,"<a href=""#image"" onclick=""image.src='images/mc..."
6,42,Sat 18,18Z,"<a href=""#image"" onclick=""image.src='images/mc..."
7,48,Sun 19,00Z,"<a href=""#image"" onclick=""image.src='images/mc..."
8,54,Sun 19,06Z,"<a href=""#image"" onclick=""image.src='images/mc..."
9,60,Sun 19,12Z,"<a href=""#image"" onclick=""image.src='images/mc..."


In [7]:
## write to text file
html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# write html to file
text_file = open("out/table.html", "w")
text_file.write(html)
text_file.close()

In [9]:
# def get_lat_lons_from_txt_file(textpts_fname):
#     ## read text file with points
#     df = pd.read_csv(textpts_fname, header=None, sep=' ', names=['latitude', 'longitude'], engine='python')
#     df['longitude'] = df['longitude']*-1
#     df = df
#     lons = df['longitude'].values
#     lats = df['latitude'].values

#     x = xr.DataArray(lons, dims=['location'])
#     y = xr.DataArray(lats, dims=['location'])

#     return x, y
# ## select transect points
# x, y = get_lat_lons_from_txt_file('../data/latlon_coast.txt')
# mclimate = mclimate.sel(lon=x, lat=y, method='nearest')
# forecast = forecast.sel(lon=x, lat=y, method='nearest')
# ## apply below loop to transect points only

In [16]:
def plot_mclimate_forecast(ds, fc, step):
    # Set up projection
    mapcrs = ccrs.PlateCarree()
    datacrs = ccrs.PlateCarree()
    
    # Set tick/grid locations
    lats = ds.lat.values
    lons = ds.lon.values
    dx = np.arange(lons.min().round(),lons.max().round()+10,10)
    dy = np.arange(lats.min().round(),lats.max().round()+10,10)
    
    ext = [-170., -120., 40., 65.]
    
    # Create figure
    fig = plt.figure(figsize=(9.5, 6.25))
    fig.dpi = 300
    fname = 'figs/ivt_mclimate_{0}_F{1}'.format(ds.init_time, step)
    fmt = 'png'
    
    nrows = 3
    ncols = 1
    
    # contour labels
    kw_clabels = {'fontsize': 7, 'inline': True, 'inline_spacing': 7, 'fmt': '%i',
                  'rightside_up': True, 'use_clabeltext': True}
    
    kw_ticklabels = {'size': 10, 'color': 'dimgray', 'weight': 'light'}
    
    ## Use gridspec to set up a plot with a series of subplots that is
    ## n-rows by n-columns
    gs = GridSpec(nrows, ncols, height_ratios=[1, 0.05, 0.05], width_ratios = [1], wspace=0.05, hspace=0.1)
    ## use gs[rows index, columns index] to access grids
    
    ###################
    ### CLIMATOLOGY ###
    ###################
    
    ax = fig.add_subplot(gs[0, 0], projection=mapcrs)
        
    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=True, right_lats=False, bottom_lons=True)
    
    # Contour Filled (mclimate values)
    data = ds.sel(step=step).ivt_mclimate.values*100.
    cmap, norm, bnds = ccmap.cmap('mclimate')
    cf = ax.contourf(lons, lats, data, transform=datacrs,
                     levels=bnds, cmap=cmap, norm=norm, alpha=0.9, extend='neither')
    
    # Contour Lines (forecast values)
    forecast = fc.sel(step=step)
    clevs = np.arange(250., 2100., 250.)
    cs = ax.contour(lons, lats, forecast.IVT, transform=datacrs,
                     levels=clevs, colors='k',
                     linewidths=0.75, linestyles='solid')
    plt.clabel(cs, **kw_clabels)
    
    # Add color bar
    cbax = plt.subplot(gs[1,0]) # colorbar axis
    cb = Colorbar(ax = cbax, mappable = cf, orientation = 'horizontal', ticklocation = 'bottom')
    cb.set_label('Model Climate Percentile Rank (xth)', fontsize=11)
    cb.ax.tick_params(labelsize=12)
    
    ts = pd.to_datetime(ds.init_time, format="%Y%m%d%H") 
    init_time = ts.strftime('%HZ %d %b %Y')
    start_date = ts - timedelta(days=45)
    start_date = start_date.strftime('%d-%b')
    end_date = ts + timedelta(days=45)
    end_date = end_date.strftime('%d-%b')
    
    ts_valid = ts + timedelta(hours=int(step))
    valid_time = ts_valid.strftime('%HZ %d %b %Y')
    
    ax.set_title('Initialized: {0}'.format(init_time), loc='left', fontsize=10)
    ax.set_title('F-{0} | Valid: {1}'.format(step, valid_time), loc='right', fontsize=10)

    
    txt = 'Relative to all 162-h GEFSv12 reforecasts initialized between {0} and {1} (2000-2019)'.format(start_date, end_date)
    ann_ax = fig.add_subplot(gs[-1, 0])
    ann_ax.axis('off')
    ann_ax.annotate(textwrap.fill(txt, 101), # this is the text
               (0, 0.3), # these are the coordinates to position the label
                textcoords="offset points", # how to position the text
                xytext=(0,-19), # distance from text to points (x,y)
                ha='left', # horizontal alignment can be left, right or center
                **kw_ticklabels)
    
    fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

    plt.close(fig)
    
    # # Show
    # plt.show()

In [ ]:
%%time
step_lst = ds.step.values

for i, step in enumerate(step_lst):
    print(step)
    plot_mclimate_forecast(ds, forecast, step=step)

6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168
174
180
186
192
198
204
210
216
222
228
234
240
